In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, to_date

# 创建一个SparkSession

In [2]:
spark = SparkSession.builder \
    .appName("Split Large CSV by Date") \
    .getOrCreate()

24/08/20 15:59:40 WARN Utils: Your hostname, owner-Super-Server resolves to a loopback address: 127.0.1.1; using 157.82.142.123 instead (on interface eno1)
24/08/20 15:59:40 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/08/20 15:59:41 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


# 读取CSV文件

In [9]:
df = spark.read.csv('/mnt/large/data/BlogWatcher/Tokyo_202307_202308/BW_Tokyo_202307_202308.csv', header=True, inferSchema=True)

Exception in thread "serve-DataFrame" java.net.SocketTimeoutException: Accept timed out
	at java.net.PlainSocketImpl.socketAccept(Native Method)
	at java.net.AbstractPlainSocketImpl.accept(AbstractPlainSocketImpl.java:409)
	at java.net.ServerSocket.implAccept(ServerSocket.java:560)
	at java.net.ServerSocket.accept(ServerSocket.java:528)
	at org.apache.spark.security.SocketAuthServer$$anon$1.run(SocketAuthServer.scala:65)


In [10]:
# 将datetime列转换为日期格式
df = df.withColumn('date', to_date(col('datetime')))

# 获取所有唯一日期
dates = df.select('date').distinct().collect()

In [ ]:
# 定义输出文件夹路径
output_folder = 'output_files'

# 按日期筛选数据并保存为单独的文件
for row in dates:
    print(row)
    break
    date = row['date'].strftime('%Y-%m-%d')
    filtered_df = df.filter(col('date') == date)
    output_file = f"{output_folder}/{date}.csv"
    filtered_df.write.csv(output_file, header=True, mode='overwrite')

print("数据切分完成！")

# 停止SparkSession
spark.stop()
